# Chapter 6 - Point Feature Detection

In the last blog article, we've seen how filters can be used to find edges in an image. However, we can also use filters to detect features in an image. 

## Template Matching
To illustrate this concept, let's have a look at an image with some simple structures on it. We can copy a small region of our image and use it as a filter / template. When we slide this filter over the image, we'll get a so called *correlation map* that indicates how well the template matched the original picture at each possible position. As we might expect, the point of best correlation is found at the brightest pixel at exactly the position where we extracted the template originally. 

![Template matching](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/template_matching.png)
*Figure 1: Template, Detected template and correlation map. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

To find the difference between the template and the image, we can use **cross correlation**, hence the name **cross correlation map**. In normalized cross correlation (NCC), we interpret pixels as vectors and take their cross product divided by the vectors lengths. If the images / vectors are equal, their NCC will be 1. If they are perpendicular, the NCC is 0 - a NCC of -1 suggest that the vectors are exact opposites of each other. 
While NCC is very robust, the problem with NCC is the computational complexity: It is computationally expensive to calculate NCC, hence it is seldomly used in practice. 

![Normalized Cross Correlation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/cross_correlation.png)
*Figure 2: Cross correlation formula. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

An alternative and much faster similarity measurement is the **Sum of Squared Differences** (SSD). SSD, as the name sais, takes the piecewise difference of each pixel value, squares it and sums the results. 

![Sum of Squared Differences](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/sum_of_squared_differences.png)
*Figure 3: Sum of Squared Differences. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

Even cheaper than SSD is the third similarity measurement, called **Sum of Absolute Differences** (SAD). It works like SSD but replaces the square with the absolute function, making it even faster to compute but also less robust. 

![Sum of Absolute Differences](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/sum_of_squared_differences.png)
*Figure 4: Sum of Absolute Differences. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

While template matching is cool, it's applications are limited to cases where we match on a pixel level. We can't match objects that are just similar to the pixel. We even fail if the overall brightness of the image changed or the rotation / orientation slightly changed. As we compare exact pixel values, we also need to have the exact pixel equivalent to the filter inside the image. 